You will need to install the following packages:
- `ipyleaflet`
- `requests`
- `xarray`
- `netcdf4`

In [ ]:
from ipyleaflet import Map, TileLayer, basemaps
from ipyleaflet.velocity import Velocity
import xarray as xr
import os
import math

In [ ]:
center = [44.33956524809713, -130.60546875000003]
zoom = 3
m = Map(
    center=center,
    zoom=zoom,
    interpolation="nearest",
    basemap=basemaps.CartoDB.DarkMatter,
)

if not os.path.exists("wind-global.nc"):
    url = "https://github.com/benbovy/xvelmap/raw/master/notebooks/wind-global.nc"
    import requests

    r = requests.get(url)
    wind_data = r.content
    with open("wind-global.nc", "wb") as f:
        f.write(wind_data)
ds = xr.open_dataset("wind-global.nc")
print (type(ds))
ds

In [ ]:
v_norm_list = []
for i in range(len(ds.lat)):
    for j in range (len(ds.lon)):
        v_norm_list.append(math.sqrt(ds.v_wind[i][j]*ds.v_wind[i][j]+ds.u_wind[i][j]*ds.u_wind[i][j]))
max_velocity = round(max(v_norm_list),1)
min_velocity = round(min(v_norm_list),1)

In [ ]:
print ('min is:', min_velocity)
print ('max is:', max_velocity)

In [ ]:
from branca.colormap import linear

mycolormap = linear.viridis

def convert_to_velocity_color_list(colormap):
    color_scale = []
    
    for i in range(len(colormap.colors)):
        rgb_tuple = tuple(str(int(x * 256)) for x in colormap.colors[i][:3])
        rgb_tuple_str = " , ".join(rgb_tuple)
        #print(rgb_tuple_str)
        rgb_str = 'rgb('+ rgb_tuple_str + ')'
        print(rgb_str)
        color_scale.append(rgb_str)

    return color_scale

mycolor_scale = convert_to_velocity_color_list(mycolormap)

In [ ]:
import ipyleaflet
from ipyleaflet import Map, ColormapControl, WidgetControl

display_options = {
    "velocityType": "Global Wind",
    "displayPosition": "bottomleft",
    "displayEmptyString": "No wind data",
}
wind = Velocity(
    data=ds,
    zonal_speed="u_wind",
    meridional_speed="v_wind",
    latitude_dimension="lat",
    longitude_dimension="lon",
    velocity_scale=0.01,
    min_velocity=min_velocity,
    max_velocity=max_velocity,
    color_scale = mycolor_scale,
    display_options=display_options,
)

m.add_layer(wind)

In [ ]:
myColormapControl = ColormapControl(
    caption="wind velocity",
    colormap_choice=mycolormap,
    value_min=wind.min_velocity,
    value_max=wind.max_velocity,
    position='topright',
    transparent_bg=False
)
m.add_control(myColormapControl)
m